## İmport İşlemleri

In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install tensorflow
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git
%pip install transformers datasets
%pip install torch
%pip install tensorflow
%pip install pyautogui
%pip install snowballstemmer
%pip install vaderSentiment
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import pyautogui
import tweepy
import csv
import sys
import json
import time
import re
import torch
from tqdm import tqdm
import warnings
import string
import pandas as pd
import ast
import nltk
from snowballstemmer import TurkishStemmer as snowTurkish
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as st
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from tweepy import Stream ,OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from trnlp import TrnlpWord
from trnlp import *
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    TextClassificationPipeline, 
    pipeline, 
    AutoModelWithLMHead,
    AutoModel,
    ElectraTokenizer, 
    TFElectraForMultipleChoice,
    TFElectraForSequenceClassification,
    BertTokenizer
    )
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from torch.utils.data import TensorDataset
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

## Twitter Veri Çekme ve Temizleme Sınıfı

In [ ]:
class getTweets:
    
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()
            
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.full_text for tweet in tweets]
            #DataSet['Tweet_Degistirilmis'] = [tweet.full_text for tweet in tweets]
            #DataSet['Tweet_Ceviri']=[tweet.full_text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            DataSet.to_csv('Datasets/tw.csv',index=False)
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,arananTweet):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=arananTweet,result_type="new", lang='tr',tweet_mode="extended")
            results=[]
            for item in cursor.items(150):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)            
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            Dataset=pd.read_csv('Datasets/tw.csv',encoding="utf-8")                   
            getTweets.TweetClear(DataSet)#Tweet Temizlemek için
            getTweets.SentenceNormalization(DataSet)#Türkçe hatalı yazılmış kelimeleri düzeltmek için
            getTweets.TweetTranslate(DataSet)#Tweetleri İngilizceye çevirmek için
            getTweets.EngWordSeparation(DataSet)#İngilizce kelime Stopwords 
            getTweets.TrWordSeparation(DataSet)#Türkçe kelime Stopwords
            getTweets.ToxicSentence(DataSet)#İngilizce Argo cümle bulmak için             
            getTweets.TrStemmed(DataSet)#Türkçe Kelime köklerini almak için
            getTweets.EngStemmed(DataSet)#İngilizce Kelime köklerini almak için
            getTweets.TextblobSentiment(DataSet)#Textblob Duygu Analizi (Kelime pozitif negatif olarak sorguluyor)           
            DataSet.to_csv('Datasets/tw.csv',index=False)

    def TextblobSentiment(DataSet):
        try:
            data=DataSet
            print ("Duygu Analizi Yapılıyor...")            
            data["TextBlobTür"] = data["Tweet_Ayrı_Eng"].apply(getTweets.getPolarity)
            data["TextBlobYoğunluk"] = data["Tweet_Ayrı_Eng"].apply(getTweets.getSubjectivity)
            data.to_csv('Datasets/textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def TweetClear(DataSet):
        try:
            data=DataSet
            print ("Tweetler Temizleniyor...")
            data["Tweet_Clear"]=data["Tweet"]
            pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-ZğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|amp[\s]+|gt;')
            for index in range(len(data["Tweet_Clear"])):        
                data["Tweet_Clear"][index]=pattern.sub(' ', data["Tweet_Clear"][index])    
                data["Tweet_Clear"][index]=data["Tweet_Clear"][index].lower()              
            data["Tweet_Clear"]=data["Tweet_Clear"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))  
                              
    def TweetTranslate(DataSet):
        try:
            data=DataSet
            print ("Tweetler Çeviriliyor...")
            data["Tweet_Eng"]=data["Tweet_Clear"]
            for index in range(len(data["Tweet_Eng"])):
                Ceviri_Blob=TextBlob(data["Tweet_Eng"][index])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                data["Tweet_Eng"][index]=Eng_Blob  
                data["Tweet_Eng"][index]=data["Tweet_Eng"][index].lower()
            print ("Tweetler Çevirildi!")                      
            return data      
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def SentenceNormalization(DataSet):
        try:
            data=DataSet
            print("Cümle Normalizasyonu Yapılıyor...")
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)
            for index in range(len(data["Tweet_Clear"])):                
                data["Tweet_Clear"][index]= normalizer.normalize(data["Tweet_Clear"][index].lower())
            print("Cümle Normalizasyonu Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TrWordSeparation(DataSet):
        try:
            data=DataSet
            print("Türkçe Kelime Ayrıştırma Yapılıyor...")
            data["Tweet_Ayrı_TR"]=data["Tweet_Clear"]
            for index in range(len(data["Tweet_Ayrı_TR"])):
                data["Tweet_Ayrı_TR"][index] = simple_token(data["Tweet_Ayrı_TR"][index], sw=stopwords)
            data["Tweet_Ayrı_TR"] = data["Tweet_Ayrı_TR"].str.join(" ")
            print("Türkçe Kelime Ayrıştırma Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))  
                  
    def EngWordSeparation(DataSet):
        try:
            data=DataSet
            print("İngilizce Kelime Ayrıştırma Yapılıyor...")
            additional  = ['sq','gt','mk','aq','qq','amk']
            stop = st.words('english')
            
            data["Tweet_Ayrı_Eng"] = data["Tweet_Eng"].str.split()           
            data["Tweet_Ayrı_Eng"] = data["Tweet_Ayrı_Eng"].apply(lambda x: [item for item in x if item not in stop])
            data["Tweet_Ayrı_Eng"] = data["Tweet_Ayrı_Eng"].apply(lambda x: [item for item in x if item not in additional])
            data["Tweet_Ayrı_Eng"] = data["Tweet_Ayrı_Eng"].str.join(" ")
            print("İngilizce Kelime Ayrıştırma Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TrStemmed(DataSet):
        try:
            data=DataSet
            data["TR_Kök"] = data["Tweet_Ayrı_TR"]
            # Kök Bulma
            stemmer = snowTurkish()

            stemmed_lists = []
            for index in tqdm(data["TR_Kök"].index):
                mini_l = []
                for text in data.loc[index]["TR_Kök"].split(" "):
                    mini_l.append(stemmer.stemWord(text))

                big_text=" "
                for char in mini_l:
                    big_text = big_text + " " + char
                stemmed_lists.append(big_text)
    
            DataSet['TR_Kök'] = stemmed_lists
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def EngStemmed(DataSet):
        try:
            data=DataSet
            data["Eng_Kök"] = data["Tweet_Ayrı_Eng"]
            # Kök Bulma
            stemmer = PorterStemmer()

            stemmed_lists = []
            for index in tqdm(data["Eng_Kök"].index):
                mini_l = []
                for text in data.loc[index]["Eng_Kök"].split(" "):
                    mini_l.append(stemmer.stem(text))

                big_text=" "
                for char in mini_l:
                    big_text = big_text + " " + char
                stemmed_lists.append(big_text)
    
            DataSet['Eng_Kök'] = stemmed_lists
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def ToxicSentence(DataSet):
        try:
            data=DataSet
            print("Toxic Cümle Analizi Yapılıyor...")
            model_path ="martin-ha/toxic-comment-model"
            tokenizer=AutoTokenizer.from_pretrained(model_path)
            model=AutoModelForSequenceClassification.from_pretrained(model_path)
            pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
            data["Tweet_Toxic"]=data["Tweet_Eng"]
            temp=data["Tweet_Clear"]
            for index in range(len(data["Tweet_Toxic"])):
                sonuc=pipeline(str(data["Tweet_Toxic"][index]))
                if sonuc[0]['label']=="toxic":
                    data["Tweet_Toxic"][index]="Toxic"
                else:
                    data["Tweet_Toxic"][index]="Toxic değil"    
            print("Toxic Cümle Analizi Yapıldı!")
            return data  
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "%75 - %100"
            elif sub >=0.5 and sub <0.75:
                return "%50 - %75"
            elif sub >=0.25 and sub <0.50:
                return "%25 - %50"
            else:
                return "0 - 25%"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

## Class'a Kullanıcı Tarafından Veri Gönderdirilen Bölüm

In [ ]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
while True:
    aramaTuru=pyautogui.confirm('Ne Tür Bir Arama Yapacaksınız?',title='Arama Türü', buttons=['Hastag','Kelime','Kullanıcı'])
    if aramaTuru=="Hastag":
        arananKelime=pyautogui.prompt(text="Aranacak Kelimeyi Girin:", title='Aranacak Kelime' , default='python')
        temp=("#"+arananKelime+" -filter:retweets")
        tw.searchTweet(arananTweet=temp)
        secim=pyautogui.confirm(text='Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break
    elif aramaTuru=='Kelime':
        arananKelime=pyautogui.prompt(text="Aranacak Kelimeyi Girin:", title='Aranacak Kelime' , default='python')
        temp=(arananKelime+" -filter:retweets")
        tw.searchTweet(arananTweet=temp)
        secim=pyautogui.confirm(text='Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break
    elif aramaTuru=='Kullanıcı':
        arananKelime=pyautogui.prompt(text="Aranacak Kelimeyi Girin:", title='Aranacak Kelime' , default='python')
        temp=("@"+arananKelime+" -filter:retweets")
        tw.searchTweet(temp)
        secim=pyautogui.confirm(text='Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break
    else:
        secim=pyautogui.confirm(text='Hatalı Seçim Yaptınız Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","Kapat"])
        if secim=="Tekrar":
            continue
        else:
            break


## OLUŞTURDUĞUMUZ TWİTTER VERİSETİ

In [ ]:
Data=pd.read_csv('Datasets/textblob_eng_duygu.csv')
Data.head()

## BERT MODEL

### bert-base-turkish-sentiment-cased

In [ ]:
BertData=pd.read_csv('Datasets/textblob_eng_duygu.csv',usecols=['Tweet','TextBlobTür','TextBlobYoğunluk','Tweet_Clear','Tweet_Eng'])
# Eğitilmiş Ağırlıkların Yüklenmesi

model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

sentiment_analysis_pipeline= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)
sentiment_result=[]
for text in tqdm(BertData["Tweet_Clear"]):
    result = sentiment_analysis_pipeline(text)[0]
    sentiment_result.append(result)
        
sentiment_result = pd.DataFrame(sentiment_result)
tweets = pd.concat([BertData,sentiment_result],axis=1)

tweets.to_csv("Datasets/Bert_Tr_Sonuc.csv",index=False)

BertData=pd.read_csv("Datasets/Bert_Tr_Sonuc.csv")
BertData.rename(columns={'label':'BertTür','score':'BertYoğunluk'}, inplace=True)
BertData["BertTür"]=BertData["BertTür"].map({'positive':'Pozitif','negative':'Negatif'})
BertData.head()

In [ ]:
labels = BertData["BertTür"].map({"Pozitif":'Pozitif',"Negatif":'Negatif'}).value_counts().index.values
sizes = BertData["BertTür"].value_counts().values

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90,explode=None)
ax1.axis('equal')
plt.show()

In [ ]:
sns.countplot(BertData["BertTür"])

### bert-base-turkish-128k-uncased

In [ ]:
Bert128Data=pd.read_csv('Datasets/textblob_eng_duygu.csv',usecols=['Tweet','TextBlobTür','TextBlobYoğunluk','Tweet_Clear','Tweet_Eng'])
# Eğitilmiş Ağırlıkların Yüklenmesi

model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased", do_lower_case=True)

sentiment_analysis_pipeline= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)
sentiment_result=[]
for text in tqdm(Bert128Data["Tweet_Clear"]):
    result = sentiment_analysis_pipeline(text)[0]
    sentiment_result.append(result)
        
sentiment_result = pd.DataFrame(sentiment_result)
tweets = pd.concat([Bert128Data,sentiment_result],axis=1)

tweets.to_csv("Datasets/Bert128_Tr_Sonuc.csv",index=False)
Bert128Data=pd.read_csv("Datasets/Bert128_Tr_Sonuc.csv")
Bert128Data.rename(columns={'label':'Bert128Tür','score':'Bert128Yoğunluk'}, inplace=True)
Bert128Data["Bert128Tür"]=Bert128Data["Bert128Tür"].map({'LABEL_0':'Pozitif','LABEL_1':'Negatif'})
Bert128Data.head()

In [ ]:
labels = Bert128Data["Bert128Tür"].map({"Pozitif":'Pozitif',"Negatif":'Negatif'}).value_counts().index.values
sizes = Bert128Data["Bert128Tür"].value_counts().values

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90,explode=None)
ax1.axis('equal')
plt.show()

In [ ]:
sns.countplot(Bert128Data["Bert128Tür"])

### paraphrase-MiniLM-L6-v2

In [ ]:
BertMiniData=pd.read_csv('Datasets/textblob_eng_duygu.csv',usecols=['Tweet','TextBlobTür','TextBlobYoğunluk','Tweet_Clear','Tweet_Eng'])

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
sentiment_result=[]
# Tokenize sentences
for text in BertMiniData["Tweet_Eng"]:
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
sentiment_result = mean_pooling(model_output, encoded_input['attention_mask'])
sentiment_result = pd.DataFrame(sentiment_result)

tweets = pd.concat([BertMiniData,sentiment_result],axis=1)
tweets.to_csv("Datasets/BertMini_Sonuc.csv",index=False)
BertMiniData=pd.read_csv("Datasets/BertMini_Sonuc.csv")
BertMiniData.head()

## ELECTRA (BERT MODEL TABANLI)

In [ ]:
ElectraData=pd.read_csv('Datasets/textblob_eng_duygu.csv',usecols=['Tweet','TextBlobTür','TextBlobYoğunluk','Tweet_Clear','Tweet_Eng'])

tokenizer = ElectraTokenizer.from_pretrained("bhadresh-savani/electra-base-emotion")
model = TFElectraForSequenceClassification.from_pretrained("bhadresh-savani/electra-base-emotion")

sentiment_result=[]
for text in tqdm(ElectraData["Tweet_Eng"]):
    inputs = tokenizer(text, return_tensors="tf")
    logits = model(**inputs).logits
    predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
    result=model.config.id2label[predicted_class_id]
    sentiment_result.append(result)
        
sentiment_result = pd.DataFrame(sentiment_result)
tweets = pd.concat([ElectraData,sentiment_result],axis=1)
tweets.to_csv("Datasets/Electra_Eng_Sonuc.csv",index=False)

ElectraData=pd.read_csv("Datasets/Electra_Eng_Sonuc.csv")
ElectraData.rename(columns={"0":"ElectraTür"}, inplace=True)
ElectraData["ElectraTür"]=ElectraData["ElectraTür"].map({"joy":'Mutlu',"anger":'Kızgın',"sadness":"Üzgün","fear":"Korku"})
ElectraData.head()


In [ ]:
labels = ElectraData["ElectraTür"].map({"Mutlu":'Mutlu',"Kızgın":'Kızgın',"Üzgün":"Üzgün","Korku":"Korku"}).value_counts().index.values
sizes = ElectraData["ElectraTür"].value_counts().values

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90,explode=None)
ax1.axis('equal')
plt.show()

In [ ]:
sns.countplot(ElectraData["ElectraTür"])

## Vader Sentiment

In [ ]:
sid_obj= SentimentIntensityAnalyzer()
VaderData=pd.read_csv('Datasets/textblob_eng_duygu.csv',usecols=['Tweet','TextBlobTür','TextBlobYoğunluk','Tweet_Clear','Tweet_Eng','Tweet_Ayrı_Eng'])
sentiment_result=[]
for text in tqdm(VaderData["Tweet_Ayrı_Eng"]):
    result = sid_obj.polarity_scores(text)
    sentiment_result.append(result)
        
sentiment_result = pd.DataFrame(sentiment_result)
tweets = pd.concat([VaderData,sentiment_result],axis=1)
tweets.to_csv("Datasets/Vader_Tr_Sonuc.csv",index=False)
VaderData=pd.read_csv("Datasets/Vader_Tr_Sonuc.csv")
VaderData.rename(columns={"neg":"VaderNegatif",'neu':'VaderNötr','pos':'VaderPozitif','compound':'VaderYoğunluk'}, inplace=True)
VaderData.head()

## Model Karşılaştırılması

In [ ]:
temp = pd.DataFrame()
tweets = pd.concat([ElectraData,VaderData,BertData,Bert128Data],axis=1)
tweets.to_csv("Datasets/Duygu_Sonuc.csv",index=False)
temp=pd.read_csv('Datasets/Duygu_Sonuc.csv',usecols=['Tweet','Tweet_Clear','Tweet_Eng','TextBlobTür','ElectraTür','VaderPozitif','VaderNegatif','VaderNötr','BertTür','Bert128Tür'])
temp.head(20)

## Son Model

In [ ]:

temp=pd.read_csv("Test/apple-twitter-sentiment-texts.csv")
def TweetClear(DataSet):
     try:
         data=DataSet
         print ("Tweetler Temizleniyor...")
         data["Text_Clear"]=data["text"]
         pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|amp[\s]+|gt;')
         for index in range(len(data["Text_Clear"])):        
             data["Text_Clear"][index]=pattern.sub(' ', data["Text_Clear"][index])    
             data["Text_Clear"][index]=data["Text_Clear"][index].lower()              
         data["Text_Clear"]=data["Text_Clear"].str.replace("[^\w\s]","")
         print ("Tweetler Temizlendi!")
         return data
     except Exception as err:
         print('Error: {}'.format(err))  

def WordSeparation(DataSet):
    try:
        data=DataSet
        print("İngilizce Kelime Ayrıştırma Yapılıyor...")
        additional  = ['sq','gt','mk','aq','qq','amk','rt']
        stop = st.words('english')
        
        data["Stopword"] = data["Text_Clear"].str.split()           
        data["Stopword"] = data["Stopword"].apply(lambda x: [item for item in x if item not in stop])
        data["Stopword"] = data["Stopword"].apply(lambda x: [item for item in x if item not in additional])
        data["Stopword"] = data["Stopword"].str.join(' ')
        print("İngilizce Kelime Ayrıştırma Yapıldı!")
        return data
    except Exception as err:
        print('Error: {}'.format(err))
  
def Stemmed(DataSet):
    try:
        data=DataSet
        data["Stemmed"] = data["Stopword"]
        # Kök Bulma
        stemmer = PorterStemmer()
        stemmed_lists = []
        for index in tqdm(data["Stemmed"].index):
            mini_l = []
            for text in data.loc[index]["Stemmed"].split(" "):
                mini_l.append(stemmer.stem(text))
            big_text=" "
            for char in mini_l:
                big_text = big_text + " " + char
            stemmed_lists.append(big_text)

        DataSet['Stemmed'] = stemmed_lists
        return data
    except Exception as err:
        print('Error: {}'.format(err))
        
TweetClear(temp)
WordSeparation(temp)
Stemmed(temp)
temp.to_csv("Test/testData.csv",index=False)
TestData=pd.read_csv("Test/testData.csv",encoding="Utf-8")
TestData.head()
    

### Tenserflow.Keras Model

Model için ön yukarıda yapıldı işlemler yapıldı.

In [ ]:
text = " ".join(review for review in TestData["Text_Clear"])
# Generate a word cloud image
wordcloud = WordCloud(background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
max_words = 500
max_len= 40

def tokenize_pad_sequences(text):
    '''
    This function tokenize the input text into sequnences of intergers and then
    pad each sequence to the same length
    '''
    # Text tokenization
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    # Transforms text to a sequence of integers
    X = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    X = pad_sequences(X, padding='post', maxlen=max_len)
    # return sequences
    return X, tokenizer

print('Before Tokenization & Padding \n',TestData["Text_Clear"][0])
X, tokenizer = tokenize_pad_sequences(TestData["Text_Clear"])
print('After Tokenization & Padding \n', X[0])



In [ ]:
vocab_size = tokenizer.document_count

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(16, activation="relu"),
    tf.keras.layers.Dense(3, activation="relu")
])

model.summary()


In [ ]:
y = pd.get_dummies(TestData["sentiment"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print('Train Set ->', X_train.shape, y_train.shape)
print('Test Set ->', X_test.shape, y_test.shape)



In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
history = model.fit(X_train, y_train, epochs=4, batch_size=64,validation_data=(X_test, y_test))

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, "acc")
plot_graphs(history, "loss")

In [ ]:
rf = RandomForestClassifier(n_estimators=550, random_state=42)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

### Sklearn RandomForest Model

Model için ön yukarıda yapıldı işlemler yapıldı.

In [ ]:
vectorizer = TfidfVectorizer( max_features=7500, min_df=1, max_df=250, stop_words=st.words('english'))
proccessed_features =vectorizer.fit_transform(TestData["Text_Clear"]).toarray()
labels = pd.get_dummies(TestData["sentiment"])
X_train, X_test, y_train, y_test =train_test_split(proccessed_features, labels, test_size=0.3,random_state=42)

In [ ]:
text_classifier=RandomForestClassifier(n_estimators=550, random_state=42)
text_classifier.fit(X_train,y_train)

In [ ]:
predictions=text_classifier.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))


### Bert Model Optimize Edilmiş

In [ ]:
BertOpData=pd.read_csv('Test/testData.csv')
vectorizer = TfidfVectorizer( max_features=7500, min_df=1, max_df=250, stop_words=st.words('english'))
proccessed_features =vectorizer.fit_transform(TestData["Text_Clear"]).toarray()
labels = pd.get_dummies(TestData["sentiment"])
X_train, X_test, y_train, y_test =train_test_split(proccessed_features, labels, test_size=0.3,random_state=42)